In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext sql 
%pylab inline
%matplotlib inline
import pylab as pl
import pandas as pd
import os, random

import pickle,time

tmstmp = time.strftime("%m-%d-%Y",time.localtime())



Populating the interactive namespace from numpy and matplotlib


In [2]:
# Set up the local source files
TOP = os.getcwd().replace('notebooks','')
LIB = TOP+'lib'
if not LIB in sys.path: sys.path.append(LIB)

DAT_DIR = TOP + 'data/'
RES_DIR = TOP + 'results/'
FIG_DIR = TOP + 'figs/'

from organtox import *

DB = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='organtox_v1')
DB.collection_names()

/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[u'chm_fp', u'ml_lr_v1', u'bio_fp', u'ml_summary_v1', u'ml_run_v1', u'tox_fp']

In [3]:
Outcomes =[u'CHR:Adrenal Gland',
 u'CHR:Bone Marrow',
 u'CHR:Brain',
 u'CHR:Eye',
 u'CHR:Heart',
 u'CHR:Kidney',
 u'CHR:Liver',
 u'CHR:Lung',
 u'CHR:Lymph Node',
 u'CHR:Mammary Gland',
 u'CHR:Pancreas',
 u'CHR:Pituitary Gland',
 u'CHR:Spleen',
 u'CHR:Stomach',
 u'CHR:Testes',
 u'CHR:Thymus',
 u'CHR:Thyroid Gland',
 u'CHR:Urinary Bladder',
 u'CHR:Uterus',
 u'MGR:Brain',
 u'MGR:Kidney',
 u'MGR:Ovary',
 u'MGR:Testes',
 u'SUB:Adrenal Gland',
 u'SUB:Bone Marrow',
 u'SUB:Brain',
 u'SUB:Heart',
 u'SUB:Kidney',
 u'SUB:Liver',
 u'SUB:Lung',
 u'SUB:Spleen',
 u'SUB:Stomach',
 u'SUB:Testes',
 u'SUB:Thymus',
 u'SUB:Thyroid Gland']
len(Outcomes)

35

In [6]:
DB.ml_lr_v1.find_one().keys()

[u'n_obs',
 u'n_neg',
 u'perf_cvt',
 u'n_ds',
 u'pred',
 u'dt_in',
 u'n_pos',
 u'dt_out',
 u'lr',
 u'perf_trn',
 u'_id']

## Solve for TP, TN, FP, FN

```
n_pos = TP + FN
n_neg = TN + FP
sens = TP/(TP+FN) = TP/n_pos
spec = TN/(TN+FP) = TN/n_neg
```

So ...

```
TP = sens*n_pos
TN = spec*n_neg
FN = n_pos-TP
FP = n_neg-TN
PPV = TP/(TP+FP)
NPV = TN/(TN+FN)
FDR = 1-PPV
```



In [28]:
def calcPerf2(n_pos,n_neg,sens,spec,sn_sd=None,sp_sd=None):
    n = n_pos+n_neg
    TP = int(sens * n_pos)
    TN = int(spec * n_neg)
    FN = n_pos - TP
    FP = n_neg - TN
    PPV= 1.0*TP/(TP+FP)
    NPV= 1.0*TN/(TN+FN)
    FDR= 1 - PPV
    
    se=sp_se=sn_se=None
    
    if sn_sd and sp_sd:
        sn_se=sn_sd/np.sqrt(n)
        sp_se=sn_sd/np.sqrt(n)
        se = np.sqrt(sn_se**2 + sp_se**2)*np.sqrt(n)
        
    return dict(#n_pos=n_pos,n_neg=n_neg,
                TP=TP,TN=TN,FP=FP,FN=FN,
                #Sens=sens,Spec=spec,
                PPV = PPV, NPV=NPV, FDR=FDR,
                se=se
               )

In [15]:
P0=DB.ml_lr_v1.find_one({},dict(_id=0,perf_cvt=1,n_pos=1,n_neg=1))
calcPerf2(P0['n_pos'],P0['n_neg'],
          P0['perf_cvt']['sens_mn'],P0['perf_cvt']['spec_mn'],
         )

{'FDR': 0.3146853146853147,
 'FN': 7,
 'FP': 45,
 'NPV': 0.8955223880597015,
 'PPV': 0.6853146853146853,
 'Sens': 0.935704288413499,
 'Spec': 0.5726854524670022,
 'TN': 60,
 'TP': 98,
 'n_neg': 105,
 'n_pos': 105,
 'se': None,
 'sn_se': None,
 'sp_se': None}

In [29]:
P0 = DB.ml_summary_v1.find_one()
calcPerf2(P0['n_pos'],P0['n_neg'],P0['sens_mn'],P0['spec_mn'],
          sn_sd=P0['sens_sd'],sp_sd=P0['spec_sd']
         )

{'FDR': 0.34615384615384615,
 'FN': 16,
 'FP': 18,
 'NPV': 0.6666666666666666,
 'PPV': 0.6538461538461539,
 'TN': 32,
 'TP': 34,
 'se': 0.27139344425831258}

In [27]:
DB.ml_summary_v1.find().distinct('n_ds')
DB.ml_summary_v1.find({'n_ds':{'$gte':24}}).count()


85440

In [30]:
DB.ml_summary_v2.drop()

for P0 in DB.ml_summary_v1.find({'n_ds':{'$gte':24}},dict(_id=0)):
    P1=calcPerf2(P0['n_pos'],P0['n_neg'],P0['sens_mn'],P0['spec_mn'],
          sn_sd=P0['sens_sd'],sp_sd=P0['spec_sd']
         )
    P0.update(P1)
    DB.ml_summary_v2.insert_one(P0)

In [32]:
DB.ml_summary_v2.find_one({'n_obs':{'$gte':200}})

{u'FDR': 0.2149532710280374,
 u'FN': 16,
 u'FP': 23,
 u'NPV': 0.8279569892473119,
 u'PPV': 0.7850467289719626,
 u'TN': 77,
 u'TP': 84,
 u'_id': ObjectId('59a39c68072e60642d33620d'),
 u'acc_mn': 0.768598126229409,
 u'acc_sd': 0.06413056255370661,
 u'bacc_mn': 0.8075466199984428,
 u'bacc_sd': 0.060566234236081075,
 u'dt_in': u'bc',
 u'dt_out': u'tox',
 u'f1_mn': 0.8008085395814939,
 u'f1_sd': 0.0626572331343638,
 u'lr': u'CART0',
 u'n_ds': 24,
 u'n_neg': 100,
 u'n_obs': 200,
 u'n_pos': 100,
 u'pred': u'MGR:Kidney',
 u'pt': u'cvt',
 u'se': 0.14448644272139846,
 u'sens_mn': 0.8422185160526867,
 u'sens_sd': 0.10216734343782254,
 u'spec_mn': 0.7728747239441982,
 u'spec_sd': 0.06997477917090296}

# Optimal classifiers / Max observations

In [36]:
# What are the top classifiers by pred, dt and ?
X = pd.DataFrame(list(DB.ml_summary_v2.find({'pred':{'$in':Outcomes},
                                             'pt':'cvt','n_obs':{'$gt':100}},
                                                dict(_id=0,pt=0,n_pos=0,n_neg=0,dt_out=0))))

Res = None
for (pred),P_i in X.groupby(['pred']):
    for (dt), P_ij in P_i.groupby(['dt_in']):
        for (lr),P_ijk in P_ij.groupby(['lr']):
            Res = pd.concat((Res,P_ijk.sort('f1_mn',ascending=False).reset_index(drop=True).head(1)))
Perf4 = Res.reset_index(drop=True)




Perf4['res2']=Perf4.apply(lambda x: "%s:%d/%s/%s:%d" % (x.pred,x.n_obs,x.lr,x.dt_in,x.n_ds),axis=1)
Perf4['res2_f1']=Perf4.apply(lambda x: "%3.2f±%3.2f" % (x.f1_mn,x.f1_sd),axis=1)
Perf4['res2_sn']=Perf4.apply(lambda x: "%3.2f±%3.2f" % (x.sens_mn,x.sens_sd),axis=1)
Perf4['res2_sp']=Perf4.apply(lambda x: "%3.2f±%3.2f" % (x.spec_mn,x.spec_sd),axis=1)
Perf4['res2_ppv']=Perf4.apply(lambda x: "%3.2f±%3.2f" % (x.PPV,x.se),axis=1)

Res = []
for p in Perf4.pred.unique():
    Res.append(Perf4.query("pred=='%s'" % p).sort('f1_mn',ascending=False).head(1))
P4 = pd.concat(Res)

In [40]:
P4.head()
P4.columns

Index([     u'FDR',       u'FN',       u'FP',      u'NPV',      u'PPV',
             u'TN',       u'TP',   u'acc_mn',   u'acc_sd',  u'bacc_mn',
        u'bacc_sd',    u'dt_in',    u'f1_mn',    u'f1_sd',       u'lr',
           u'n_ds',    u'n_obs',     u'pred',       u'se',  u'sens_mn',
        u'sens_sd',  u'spec_mn',  u'spec_sd',     u'res2',  u'res2_f1',
        u'res2_sn',  u'res2_sp', u'res2_ppv'],
      dtype='object')

In [47]:
P4[['res2','pred','dt_in','n_obs','lr','res2_f1','res2_sn','res2_sp',
    'res2_ppv']].reset_index(drop=True).to_csv(DAT_DIR+'ml-organtox-perf-aug-26-2017.csv')